# Train VAE model

This notebook will first try to train the current VAE model before modifying the loss function to work with count data

In [ ]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2
import os
from ponyo import utils, train_vae_modules

In [ ]:
# Set seeds to get reproducible VAE trained models
train_vae_modules.set_all_seeds()

In [ ]:
base_dir = os.path.abspath(os.path.join(os.getcwd(), "../"))

# Read in config variables
config_filename = os.path.abspath(
    os.path.join(base_dir, "configs", "config_human_general.tsv")
)

params = utils.read_config(config_filename)

dataset_name = params["dataset_name"]

normalized_compendium_filename = params["normalized_compendium_filename"]

In [ ]:
# Create VAE directories if needed
output_dirs = [
    os.path.join(base_dir, dataset_name, "models"),
    os.path.join(base_dir, dataset_name, "logs"),
]

NN_architecture = params["NN_architecture"]

# Check if NN architecture directory exist otherwise create
for each_dir in output_dirs:
    sub_dir = os.path.join(each_dir, NN_architecture)
    os.makedirs(sub_dir, exist_ok=True)

In [ ]:
# Train VAE on new compendium data
train_vae_modules.train_vae(config_filename, normalized_compendium_filename)